In [ ]:
import os
import pandas as pd
import mailbox
import bs4
path = '../Takeout/Mail/All mail Including Spam and Trash.mbox'
def remove_html(text):
soup = BeautifulSoup(text, "html.parser", from_encoding="iso-8859-1")
html_free=soup.get_text()
return html_free
def get_html_text(html):
try:
return bs4.BeautifulSoup(html, 'lxml').body.get_text(' ', strip=True)
except AttributeError: # message contents empty
return None
class GmailMboxMessage():
def __init__(self, email_data):
if not isinstance(email_data, mailbox.mboxMessage):
raise TypeError('Variable must be type mailbox.mboxMessage')
self.email_data = email_data
def parse_email(self):
email_labels = self.email_data['X-Gmail-Labels']
email_date = self.email_data['Date']
email_from = self.email_data['From']
email_to = self.email_data['To']
email_subject = self.email_data['Subject']
email_text = self.read_email_payload()
temp = ''
if type(email_text) == str:
for i in email_text:
temp+=i
email_text = temp
res =
{'labels':email_labels,'date':email_date,'email_from':email_from,'emial_to':email_to,'sub
ject':email_subject,'text':email_text}
return res
def read_email_payload(self):
email_payload = self.email_data.get_payload()
if self.email_data.is_multipart():
email_messages = list(self._get_email_messages(email_payload))
else:
email_messages = [email_payload]
return [self._read_email_text(msg) for msg in email_messages]
def _get_email_messages(self, email_payload):
for msg in email_payload:
if isinstance(msg, (list,tuple)):
for submsg in self._get_email_messages(msg):
yield submsg
elif msg.is_multipart():
for submsg in self._get_email_messages(msg.get_payload()):
yield submsg
else:
yield msg
def _read_email_text(self, msg):
content_type = 'NA' if isinstance(msg, str) else msg.get_content_type()
encoding = 'NA' if isinstance(msg, str) else msg.get('Content-Transfer-Encoding',
'NA')
if 'text/plain' in content_type and 'base64' not in encoding:
msg_text = msg.get_payload()
elif 'text/html' in content_type and 'base64' not in encoding:
msg_text = get_html_text(msg.get_payload())
elif content_type == 'NA':
msg_text = get_html_text(msg)
else:
msg_text = None
return (msg_text)
########################################################
import time
start_time = time.time()
mbox_obj = mailbox.mbox(path)
df = pd.DataFrame()
for idx,i in enumerate(mbox_obj):
email = GmailMboxMessage(i)
df = df.append(email.parse_email(),ignore_index=True)
if idx % 100==0:
print(idx)
df.to_csv('emails.csv')
print(time.time()-start_time)
df1 = pd.read_csv('emails.csv')
df1['class_labels']=''
def joiner(temp_list):
temp=''
for i in temp_list:
temp+=i
return temp
def stripper(row):
mylist
=['Inbox','Unread','Category','IMAP_NotJunk','IMAP_$NotJunk','Opened','Important','IM
AP_$Junk','Category','']
try:
i = re.split("\s|(?<!\d)[,.](?!\d)",row['labels'])
Kaggle Datasets ETL
for rec in mylist:
if rec in i:
i.remove(rec)
if 'Spam' in i:
return 'Spam'
elif 'Travel' in i:
return 'Travel'
elif 'Purchases' in i:
return 'Purchases'
else:
return joiner(i)
except:
pass
df1['class_labels']= df1.apply(stripper,axis=1)
#df1['class_labels'].value_counts()
#df1.drop(['target_label','new_labels'],axis=1,inplace=True)
class_list =['Promotions','Updates','Social','Personal','Purchases','Spam','Travel']
df_final = df1[df1['class_labels']